# Crawl, load, and split D&C People

In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

In [ ]:
import os
from urllib.parse import urljoin, urlparse

from bs4 import BeautifulSoup

from models.crawl_utils import get_page, save_page
from datetime import datetime

from models.load_dc_people import load_dc_people
from models.load_utils import Loader, load_docs_from_jsonl, save_docs_to_jsonl

from models.split_model import MarkdownSyntacticEmbeddingSplitter

## Crawl

In [ ]:
# config
host = 'https://doctrineandcovenantscentral.org/people-of-the-dc'
source = 'dc_people'  # use this name in the directories
crawl_dir = f'../data/raw/{source}'  # base_dir is now crawl_dir
bs_parser = 'html.parser'
delay_seconds = 5

In [ ]:
if not os.path.exists(crawl_dir):
    os.makedirs(crawl_dir)

In [ ]:
def extract_hrefs(html):
    hrefs = []

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # Find all div tags with the class 'views-field-title'
    anchor_tags = soup.find_all("a", class_ = "elementor-flip-box__button")

    # Extract href attribute from each anchor tag and append to the list
    for anchor in anchor_tags:
        href = anchor.get('href')
        if href:
            hrefs.append(href)

    return hrefs

In [ ]:
def get_path(url):
    if not url.endswith('/'):
        url += '/'
    path_components = urlparse(url).path.split('/')
    return os.path.join(crawl_dir, f"{path_components[-2]}.json")
    

In [ ]:
dc_people_links = []

status_code, html = get_page(host, delay_seconds)
if status_code != 200:
    print("Error!", status_code)
hrefs = extract_hrefs(html)
if len(hrefs) == 0:
    print(hrefs)
else:
    dc_people_links.extend(hrefs)
print(dc_people_links[:5])
len(dc_people_links)

In [ ]:
for url in dc_people_links:
    path_file =  get_path(url)
    print(path_file)
    if os.path.exists(path_file):
        continue
    status_code, html = get_page(url, delay_seconds)
    if status_code != 200:
        print("Error!", status_code , url)
        continue
    save_page(path_file,url,html)
    
print("End")

## Load

In [ ]:
# configure
load_dir = f'../data/load/{source}/'
today = datetime.today().strftime('%Y-%m-%d')
load_filename = os.path.join(load_dir, f"{today}.jsonl")

if not os.path.exists(load_dir):
    os.makedirs(load_dir)

In [ ]:
loader = Loader(load_dc_people, crawl_dir)
docs = loader.load(verbose=True)
len(docs)

In [ ]:
print(docs[0].metadata)
print(docs[0].page_content)

In [ ]:
save_docs_to_jsonl(docs, load_filename)

## Split

In [ ]:
# configure
# input_path is now load_filename, output_dir is now split_dir, and output filename is now split_filename
split_dir = f'../data/split/{source}/'
today = datetime.today().strftime('%Y-%m-%d')
# output filename is now split_filename
split_filename = os.path.join(split_dir, f"{today}.jsonl")

if not os.path.exists(split_dir):
    os.makedirs(split_dir)

In [ ]:
docs = load_docs_from_jsonl(load_filename)
len(docs)

In [ ]:
text_splitter = MarkdownSyntacticEmbeddingSplitter()

In [ ]:
splits = text_splitter.split_documents(docs, verbose=True)
len(splits)

In [ ]:
for ix, split in enumerate(splits[:10]):
    print(ix, split.metadata)
    print(split.page_content)
    print("\n!!! SPLIT !!!\n")

In [ ]:
save_docs_to_jsonl(splits, split_filename)